In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

데이터 불러오는 코드 작성- > 데이터 np.float64

In [ ]:
column_names = ['video_id', 'activity', 'frame', 
                'd_vector_1_x', 'd_vector_1_y',
                'd_vector_2_x', 'd_vector_2_y',
                'd_vector_3_x', 'd_vector_3_y',
                'd_vector_4_x', 'd_vector_4_y',
                'd_vector_5_x', 'd_vector_5_y',
                'd_vector_6_x', 'd_vector_6_y',
                'd_vector_7_x', 'd_vector_7_y',
                'd_vector_8_x', 'd_vector_8_y',
                'd_vector_9_x', 'd_vector_9_y',
                'd_vector_10_x', 'd_vector_10_y',
                'd_vector_11_x', 'd_vector_11_y',
                'd_vector_12_x', 'd_vector_12_y',
                'd_vector_13_x', 'd_vector_13_y',
                'd_vector_14_x', 'd_vector_14_y',
                'r_vector_1_x', 'r_vector_1_y',
                'r_vector_2_x', 'r_vector_2_y',
                'r_vector_3_x', 'r_vector_3_y',
                'r_vector_4_x', 'r_vector_4_y',
                'r_vector_5_x', 'r_vector_5_y',
                'r_vector_6_x', 'r_vector_6_y',
                'r_vector_7_x', 'r_vector_7_y',
                'r_vector_8_x', 'r_vector_8_y',
                'r_vector_9_x', 'r_vector_9_y',
                'r_vector_10_x', 'r_vector_10_y',
                'r_vector_11_x', 'r_vector_11_y',

                ]

df = pd.read_csv('데이터 path', header=None, names=column_names)

df.dropna(axis=0, how='any', inplace=True)

In [ ]:
sns.countplot(x = 'activity',
              data = df,
              order = df.activity.value_counts().index);

In [ ]:
sns.countplot(x = 'video_id',
              data = df,
              palette=[sns.color_palette()[0]],
              order = df.user_id.value_counts().index);
plt.title("Records per user");

In [ ]:
# 비디오 수 보고 다시 판단
df_train = df[df['video_id'] <= 330]
df_test = df[df['video_id'] > 330]

In [ ]:
scale_columns = [ 'd_vector_1_x', 'd_vector_1_y',
                'd_vector_2_x', 'd_vector_2_y',
                'd_vector_3_x', 'd_vector_3_y',
                'd_vector_4_x', 'd_vector_4_y',
                'd_vector_5_x', 'd_vector_5_y',
                'd_vector_6_x', 'd_vector_6_y',
                'd_vector_7_x', 'd_vector_7_y',
                'd_vector_8_x', 'd_vector_8_y',
                'd_vector_9_x', 'd_vector_9_y',
                'd_vector_10_x', 'd_vector_10_y',
                'd_vector_11_x', 'd_vector_11_y',
                'd_vector_12_x', 'd_vector_12_y',
                'd_vector_13_x', 'd_vector_13_y',
                'd_vector_14_x', 'd_vector_14_y',
                'r_vector_1_x', 'r_vector_1_y',
                'r_vector_2_x', 'r_vector_2_y',
                'r_vector_3_x', 'r_vector_3_y',
                'r_vector_4_x', 'r_vector_4_y',
                'r_vector_5_x', 'r_vector_5_y',
                'r_vector_6_x', 'r_vector_6_y',
                'r_vector_7_x', 'r_vector_7_y',
                'r_vector_8_x', 'r_vector_8_y',
                'r_vector_9_x', 'r_vector_9_y',
                'r_vector_10_x', 'r_vector_10_y',
                'r_vector_11_x', 'r_vector_11_y'
]

scaler = RobustScaler()

scaler = scaler.fit(df_train[scale_columns])

df_train.loc[:, scale_columns] = scaler.transform(
  df_train[scale_columns].to_numpy()
)

df_test.loc[:, scale_columns] = scaler.transform(
  df_test[scale_columns].to_numpy()
)

In [ ]:
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

In [ ]:

TIME_STEPS = 80
STEP = 80 #window 부분 - 파라미터 조정하면서 봐야할듯....

X_train, y_train = create_dataset(
    df_train[[ 'd_vector_1_x', 'd_vector_1_y',
                'd_vector_2_x', 'd_vector_2_y',
                'd_vector_3_x', 'd_vector_3_y',
                'd_vector_4_x', 'd_vector_4_y',
                'd_vector_5_x', 'd_vector_5_y',
                'd_vector_6_x', 'd_vector_6_y',
                'd_vector_7_x', 'd_vector_7_y',
                'd_vector_8_x', 'd_vector_8_y',
                'd_vector_9_x', 'd_vector_9_y',
                'd_vector_10_x', 'd_vector_10_y',
                'd_vector_11_x', 'd_vector_11_y',
                'd_vector_12_x', 'd_vector_12_y',
                'd_vector_13_x', 'd_vector_13_y',
                'd_vector_14_x', 'd_vector_14_y',
                'r_vector_1_x', 'r_vector_1_y',
                'r_vector_2_x', 'r_vector_2_y',
                'r_vector_3_x', 'r_vector_3_y',
                'r_vector_4_x', 'r_vector_4_y',
                'r_vector_5_x', 'r_vector_5_y',
                'r_vector_6_x', 'r_vector_6_y',
                'r_vector_7_x', 'r_vector_7_y',
                'r_vector_8_x', 'r_vector_8_y',
                'r_vector_9_x', 'r_vector_9_y',
                'r_vector_10_x', 'r_vector_10_y',
                'r_vector_11_x', 'r_vector_11_y'
]],
    df_train.activity,
    TIME_STEPS,
    STEP
)

X_test, y_test = create_dataset(
    df_test[[ 'd_vector_1_x', 'd_vector_1_y',
                'd_vector_2_x', 'd_vector_2_y',
                'd_vector_3_x', 'd_vector_3_y',
                'd_vector_4_x', 'd_vector_4_y',
                'd_vector_5_x', 'd_vector_5_y',
                'd_vector_6_x', 'd_vector_6_y',
                'd_vector_7_x', 'd_vector_7_y',
                'd_vector_8_x', 'd_vector_8_y',
                'd_vector_9_x', 'd_vector_9_y',
                'd_vector_10_x', 'd_vector_10_y',
                'd_vector_11_x', 'd_vector_11_y',
                'd_vector_12_x', 'd_vector_12_y',
                'd_vector_13_x', 'd_vector_13_y',
                'd_vector_14_x', 'd_vector_14_y',
                'r_vector_1_x', 'r_vector_1_y',
                'r_vector_2_x', 'r_vector_2_y',
                'r_vector_3_x', 'r_vector_3_y',
                'r_vector_4_x', 'r_vector_4_y',
                'r_vector_5_x', 'r_vector_5_y',
                'r_vector_6_x', 'r_vector_6_y',
                'r_vector_7_x', 'r_vector_7_y',
                'r_vector_8_x', 'r_vector_8_y',
                'r_vector_9_x', 'r_vector_9_y',
                'r_vector_10_x', 'r_vector_10_y',
                'r_vector_11_x', 'r_vector_11_y'
]],
    df_test.activity,
    TIME_STEPS,
    STEP
)

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(y_train)

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

In [ ]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
      keras.layers.LSTM(
          units=128,
          input_shape=[X_train.shape[1], X_train.shape[2]]
      )
    )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['acc']
)

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend();

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test)

heatmap 출력

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_cm(y_true, y_pred, class_names):
  cm = confusion_matrix(y_true, y_pred)
  fig, ax = plt.subplots(figsize=(18, 16)) 
  ax = sns.heatmap(
      cm, 
      annot=True, 
      fmt="d", 
      cmap=sns.diverging_palette(220, 20, n=7),
      ax=ax
  )

  plt.ylabel('Actual')
  plt.xlabel('Predicted')
  ax.set_xticklabels(class_names)
  ax.set_yticklabels(class_names)
  b, t = plt.ylim() 
  b += 0.5 
  t -= 0.5 
  plt.ylim(b, t) 
  plt.show() 

In [ ]:
plot_cm(
  enc.inverse_transform(y_test),
  enc.inverse_transform(y_pred),
  enc.categories_[0]
)